In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import torch
from actuallysparse import converter
from warnings import filterwarnings
from torch.autograd.profiler import profile

filterwarnings("ignore")

device = "cuda" if torch.cuda.is_available() else "cpu"
model_loader_very_pruned = lambda: torch.load("../.weights/full/very_pruned", map_location=torch.device('cpu')).eval()
model_loader_pruned = lambda: torch.load("../.weights/full/pruned", map_location=torch.device('cpu')).eval()
model_loader_pretrained = lambda: torch.load("../.weights/full/pretrained", map_location=torch.device('cpu')).eval()

In [3]:
dummy_input = torch.ones(1,3,244,244)
dense = model_loader_very_pruned().to(device).eval()
coo = converter.convert_model(model_loader_very_pruned(), torch.nn.Linear, "coo").to(device)
coo.train(False)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [4]:
with profile(profile_memory=True, record_shapes=True) as prof:
        dense(dummy_input)

print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=20))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     aten::conv2d         0.06%      85.000us        82.85%     111.749ms      13.969ms      33.46 Mb           0 b             8  
                aten::convolution         0.18%     245.000us        82.78%     111.664ms      13.958ms      33.46 Mb           0 b             8  
               aten::_convolution         0.11%     148.000us        82.60%     111.419ms      13.927ms      33.46 Mb           0 b             8  
         aten::mkldnn_convolution        82.31%     111.022ms        82.49%     111.271ms      13.909ms      33.

In [5]:
with profile(profile_memory=True, record_shapes=True) as prof:
        coo(dummy_input)

print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=20))

--------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                              Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
--------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                      aten::conv2d         0.04%      50.000us        80.13%     110.706ms      13.838ms      33.46 Mb           0 b             8  
                                 aten::convolution         0.13%     181.000us        80.10%     110.656ms      13.832ms      33.46 Mb           0 b             8  
                                aten::_convolution         0.07%      96.000us        79.96%     110.475ms      13.809ms      33.46 Mb           0 b             8  
          